In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

from dm import Country, DemographicModeling
from dm.utils import get_countries, add_enrich_aliases

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import canada_modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [3]:
can = Country('CAN')

can

<dm.Country - CAN (local)>

In [4]:
van_df = can.cmacas.get('vancouver')

van_df

,ID,NAME,SHAPE
0,933,Vancouver,"{""rings"": [[[-122.89240540549281, 49.087271790..."


In [5]:
mapwgt = van_df.spatial.plot()

mapwgt

MapView(layout=Layout(height='400px', width='100%'))

In [7]:
rona_df = can.business.get_by_name('RONA', van_df)
rona_df = rona_df[rona_df['CONAME'] == 'RONA'].copy()

rona_df.spatial.plot(mapwgt)

rona_df

,OBJECTID,LOCNUM,CONAME,STREET,FSA,CITY,PROV,PROV_NAME,POSTCD,NAICS,...,FRNCOD,ISCODE,LOC_NAME,STATUS,SCORE,SOURCE,SHAPE,id,brand_name,brand_name_category
1,1082511,734334232,RONA,Canoe Pass,V4M,Tsawwassen,BC,British Columbia,V4M 0B2,44411002,...,R,,StreetAddress,M,95.76,INFOGROUP,"{""x"": -123.08721383163152, ""y"": 49.04037562312...",734334232,RONA,RONA
3,705042,47496609,RONA,Elmbridge,V6X,Richmond,BC,British Columbia,V6X 3J7,44411002,...,R,,StreetAddress,M,100.00,INFOGROUP,"{""x"": -123.14639519918047, ""y"": 49.17249727408...",47496609,RONA,RONA
18,213405,20690202,RONA,3rd,V7P,North Vancouver,BC,British Columbia,V7J 1B8,44411002,...,R,,StreetAddress,M,89.19,INFOGROUP,"{""x"": -123.10664530032206, ""y"": 49.32028244129...",20690202,RONA,RONA
19,506116,41514332,RONA,1st,V7M,North Vancouver,BC,British Columbia,V7P 1A4,44411002,...,R,,StreetAddress,M,100.00,INFOGROUP,"{""x"": -123.09873986710704, ""y"": 49.31805595626...",41514332,RONA,RONA
21,279327,25950635,RONA,King George,V4P,Surrey,BC,British Columbia,V4P 1B8,44411002,...,R,,PointAddress,M,100.00,INFOGROUP,"{""x"": -122.81060895875102, ""y"": 49.05932610001...",25950635,RONA,RONA
22,213406,20690293,RONA,King George,V3W,Surrey,BC,British Columbia,V3W 5A1,44411002,...,R,,PointAddress,M,100.00,INFOGROUP,"{""x"": -122.84563192797422, ""y"": 49.12889930571...",20690293,RONA,RONA
24,213407,20690301,RONA,Edmonds,V3N,Burnaby,BC,British Columbia,V3N 1A6,44411002,...,R,,PointAddress,M,100.00,INFOGROUP,"{""x"": -122.9520458211351, ""y"": 49.217894272908...",20690301,RONA,RONA
27,278597,25899790,RONA,Lougheed,V2X,Maple Ridge,BC,British Columbia,V2X 2R4,44411002,...,R,,PointAddress,M,100.00,INFOGROUP,"{""x"": -122.63303946342596, ""y"": 49.21715798199...",25899790,RONA,RONA
28,213404,20690194,RONA,Lebleu,V3K,Coquitlam,BC,British Columbia,V3K 6X4,44411002,...,R,,StreetAddress,M,100.00,INFOGROUP,"{""x"": -122.86762367607292, ""y"": 49.24806780424...",20690194,RONA,RONA
31,882661,59298430,RONA,Coast Meridian,V3B,Port Coquitlam,BC,British Columbia,V3B 3N3,44411002,...,R,,PointAddress,M,100.00,INFOGROUP,"{""x"": -122.75616109111519, ""y"": 49.27139040220...",59298430,RONA,RONA


In [9]:
comp_df = can.business.get_competition(rona_df, van_df, local_threshold=1)

comp_df.spatial.plot(mapwgt)

comp_df

,OBJECTID,LOCNUM,CONAME,STREET,FSA,CITY,PROV,PROV_NAME,POSTCD,NAICS,...,FRNCOD,ISCODE,LOC_NAME,STATUS,SCORE,SOURCE,SHAPE,id,brand_name,brand_name_category
0,365507,32016495,Home Depot,Sweden,V6V,Richmond,BC,British Columbia,V6V 2W8,44411002,...,H,,PointAddress,M,100.0,INFOGROUP,"{""x"": -123.08257114600376, ""y"": 49.19408195921...",32016495,Home Depot,Home Depot
1,687992,46209771,Home Depot,Main,V7T,West Vancouver,BC,British Columbia,V7T 2Z3,44411002,...,H,,StreetAddress,M,100.0,INFOGROUP,"{""x"": -123.14103566334582, ""y"": 49.32576087871...",46209771,Home Depot,Home Depot
2,771603,53085346,Home Depot,Cambie,V5Y,Vancouver,BC,British Columbia,V5Z 2T8,44411002,...,H,,StreetAddress,M,100.0,INFOGROUP,"{""x"": -123.11474668035514, ""y"": 49.26422870194...",53085346,Home Depot,Home Depot
3,1093985,738425881,Lowe's Home Improvement,12th,V5M,Vancouver,BC,British Columbia,V5M 4W3,44411002,...,L,,PointAddress,M,100.0,INFOGROUP,"{""x"": -123.04896009843839, ""y"": 49.25873331171...",738425881,Lowe's Home Improvement,Lowe's Home Improvement
4,370159,32376121,Home Depot,Terminal,V5T,Vancouver,BC,British Columbia,V6A 4G4,44411002,...,H,,PointAddress,M,100.0,INFOGROUP,"{""x"": -123.08270397090162, ""y"": 49.26963334380...",32376121,Home Depot,Home Depot
5,859954,58385584,Home Depot,Henning,V5C,Burnaby,BC,British Columbia,V5C 6M2,44411002,...,H,,StreetAddress,M,100.0,INFOGROUP,"{""x"": -123.01787733588401, ""y"": 49.26390343711...",58385584,Home Depot,Home Depot
6,353092,31099765,Home Depot,120,V3W,Surrey,BC,British Columbia,V3W 3M9,44411002,...,H,,StreetAddress,M,100.0,INFOGROUP,"{""x"": -122.89021813443841, ""y"": 49.13644572800...",31099765,Home Depot,Home Depot
7,924259,60974029,Lowe's Home Improvement,Tanaka,V3M,New Westminster,BC,British Columbia,V3M 0G2,44411002,...,L,,StreetAddress,M,100.0,INFOGROUP,"{""x"": -122.9530460332202, ""y"": 49.186469604749...",60974029,Lowe's Home Improvement,Lowe's Home Improvement
8,279226,25943804,Home Depot,110,V3V,Surrey,BC,British Columbia,V3V 3J7,44411002,...,H,,PointAddress,M,100.0,INFOGROUP,"{""x"": -122.87060933920908, ""y"": 49.20268840413...",25943804,Home Depot,Home Depot
9,353947,31162274,Home Depot,United,V3K,Coquitlam,BC,British Columbia,V3K 6Z1,44411002,...,H,,PointAddress,M,100.0,INFOGROUP,"{""x"": -122.84123196264458, ""y"": 49.22812285778...",31162274,Home Depot,Home Depot


In [10]:
rona_fc = rona_df.spatial.to_featureclass(gdb_int/'biz_rona')

rona_fc

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\biz_rona'

In [11]:
comp_fc = comp_df.spatial.to_featureclass(gdb_int/'biz_comp')

comp_fc

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\biz_comp'